# (1) Read data
pip install jieba

In [ ]:
#pip install jieba

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#設定matplot可以直接在Notebook中畫出，不需要執行plt.plot()
%matplotlib inline 

# 設定 Seaborn 的顯示樣式
# https://seaborn.pydata.org/generated/seaborn.set_style.html
sns.set_style('whitegrid')

#載入資料
#data = pd.read_csv("data/Op_Dcard_markeup.csv")
data = pd.read_csv("data/Dcard_makeup.csv")
data
print(data.shape)  #type(pandas.core.frame.DataFrame)
data.head(3)

# (2) 格式轉換

- transform Type(pandas.core.series.Series) to list
- transform Type(list) to str
- addition: https://blog.csdn.net/Zx_whu/article/details/61926655

In [ ]:
dt_list = data['content'].tolist()   # transform Type(pandas.core.series.Series) to list
for i in range(len(dt_list)):
    print(dt_list[i])
    print('====================')

In [ ]:
#dt_list[0]      #Content 第一篇文章
#dt_list[1]     #Content 第二篇文章
#dt_list[87]    #最後一篇文章 #type(str)

# (3) Jeiba 語句分析

- https://github.com/ldkrsi/jieba-zh_TW
- https://blog.kennycoder.io/2020/02/12/Python-知名Jieba中文斷詞工具教學/

jieba.cut生成的是一个生成器，generator，也就是可以通过for循环来取里面的每一个词。
- 全模式: 把句子中所有的可以成詞的詞語都斷出来
- jieba.cut(sentence, cut_all=True)
- 精確模式 : 將句子最精確的切開，適合文本分析
- jieba.cut(sentence, cut_all=False)
- 搜索引擎模式: 在精確模式的基礎上，對長的詞語再次切分，提高召回率
- jieba.cut_for_search(sentence)

jieba.lcut 直接生成的就是一个list
- jieba.lcut(news)

## 負面關鍵字過濾前

In [ ]:
import jieba
import jieba.analyse
jieba.load_userdict('data/usedict.txt')   #adjust

#dt_list = data['content'].tolist()   # transform Type(pandas.core.series.Series) to list

for i in range(1):
    #print(dt_list[i])
    seg_list = jieba.lcut(dt_list[i])
    #data['content'][i] = seg_list
    print(seg_list)
    print('====================')


## 負面關鍵字過濾後

In [ ]:
stopWords=[]
remainderWords=[]
jieba.load_userdict('data/usedict.txt')   #adjust

with open('data/stopWords.txt','r',encoding='utf-8') as file:
    for w in file.readlines():
        w=w.strip()
        stopWords.append(w)

for i in range(len(dt_list)):
    seg_list = jieba.lcut(dt_list[i],cut_all=False)
    remainderWords = list(filter(lambda a: a not in stopWords and a != '\n', seg_list))
    data['content'][i] = remainderWords
    print(remainderWords)
    print('====================')
    

In [ ]:
#data['content'][0]
data.head(2)

In [ ]:
print(data['content'][0])

# (4) Jeiba 分佈分析
- TF-IDF算法 https://zhuanlan.zhihu.com/p/95358646

In [ ]:
dt_list = data['content'].tolist()
dt_str= ''.join(map(str,dt_list))
dt_str  #type string

In [ ]:
tags = jieba.analyse.extract_tags(dt_str, topK=30, withWeight=True)
for tag in tags:
    print('tf-idf:', round(tag[1],5),'| word:', tag[0])

# (5) apyori 關聯性分析
- https://pypi.org/project/apyori/
- https://chwang12341.medium.com/machine-learning-關聯分析-apriori演算法-詳細解說啤酒與尿布的背後原理-python實作-scikit-learn一步一步教學-76b7778f8f34

- pip install apyori
- pip install mlxtend

In [ ]:
data.head(2)

In [ ]:
print(data['content'][0],data['content'][1],data['content'][2])

In [ ]:
# dt =  [['T-Shirt','Pants','Jeans','Jersy','Socks','Basketball','Bottle','Shorts'],
#        ['T-Shirt','Jeans'],
#        ['Jersy','Basketball','Socks','Bottle'],
#        ['Jeans','Pants','Bottle'],
#        ['Shorts','Basketball'],
#        ['Shorts','Jersy'],
#        ['T-Shirt'],
#        ['Basketball','Jersy'],
#       ]

# dt =  [data['content'][0],data['content'][1]
#       ]

In [ ]:
from apyori import apriori
## Data 自行定義數據
dt =  [data['content'][0],data['content'][1],data['content'][2],data['content'][3],data['content'][4],
       data['content'][5],data['content'][6],data['content'][7],data['content'][8],data['content'][9]
       
      ] 
                    
association_rules = apriori(dt, min_support=0.2,   # 總共200筆交易數據，香腸出現20筆, SUP為 50/200 = 1/4,
                            min_confidence=0.2,    # 兩物品同時出現的條件機率，出現商品A的情況下，出現商品B的機率
                            min_lift=2,            # 數據間的關聯性，正相關>1 : 不相關=1 : 負相關<1
                            max_length=2)
                    
association_results = list(association_rules)
#print(association_results)


for product in association_results:
    pair = product[0] 
    products = [x for x in pair]
    #print(products)
    print("Rule: " + products[0] + " → " + products[1])
    print("Support: " + str(product[1]))
    print("Lift: " + str(product[2][0][3]))
    print("==================================")
    #print(product) # ex. RelationRecord(items=frozenset({'Basketball', 'Socks'}), support=0.25, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Basketball'}), items_add=frozenset({'Socks'}), confidence=0.5, lift=2.0), OrderedStatistic(items_base=frozenset({'Socks'}), items_add=frozenset({'Basketball'}), confidence=1.0, lift=2.0)])
    #print(pair)    # ex. frozenset({'Basketball', 'Socks'})

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
## Data Defined
dt = {'Transaction ID': [1,2,3,4,5,6,7,8],
      'Items':[['T-Shirt','Pants','Jeans','Jersy','Socks','Basketball','Bottle','Shorts'],
               ['T-Shirt','Jeans'],
               ['Jersy','Basketball','Socks','Bottle'],
               ['Jeans','Pants','Bottle'],
               ['Shorts','Basketball'],
               ['Shorts','Jersy'],
               ['T-Shirt'],
               ['Basketball','Jersy'],              ]}

data = pd.DataFrame(dt)
pd.options.display.max_colwidth = 100        # 讓 DataFrame 能呈現的寬度大一點
data_id = data.drop('Items', 1)              # 轉成數值編碼，目前都是字串的組合
data_items = data.Items.str.join(',')
data_items = data_items.str.get_dummies(',') # 轉成數值
data = data_id.join(data_items)              ## 接上Transaction ID

## 計算支持度 Support
Support_items = apriori(data[['T-Shirt','Pants','Jeans','Jersy','Socks','Basketball','Bottle','Shorts']], 
                        min_support=0.20, use_colnames = True)

## 計算關聯規則 Association Rule
Association_Rules = association_rules(Support_items, metric = 'lift', min_threshold=1)
Association_Rules